In [1]:
from sklearn.neighbors import NearestNeighbors
from sklearn.base import BaseEstimator

In [34]:
class EstimatorKNN(BaseEstimator):
    def __init__(self, n_neighbors=3):
        self.estimator = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree')

    def fit(self, x):
        self.estimator.fit(x)
        return self 

    def predict(self, x):
        return [1.0]*len(x) 
    
    def decision_function(self, x, y=None):
        distances, indices = self.estimator.kneighbors(x)
        outlier_level = distances[:,-1]
        return outlier_level

    def score(self, x, y=None):
        scores = self.decision_function(x)
        return sum(scores)/len(scores)

    def get_params(self, deep=True):
        return {'param1': self.param1, 'param2': self.param2}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self,parameter, value)
        return self

In [32]:
a = [[1,1],[2,1],[3,0],[4,0],[5,0],[6,1]]
[_a[0] for _a in a if _a[1] == 1]

[1, 2, 6]

In [35]:
#import estimator_knn as knn
import sys
import notebookutil as nbu
sys.meta_path.append(nbu.NotebookFinder())
import datasets
import pandas as pd
import numpy as np

def eval01():
    ds = datasets.load('data/raw/01_0[12]_01_*')
    column = ['rssi.a','rssi.b',['linear_accel[0]','linear_accel[1]','linear_accel[2]']]
    data_n, data_o = datasets.get_normalized_data(ds,
                                                  title_n='01_01_01_4F実験室_XperiaZ3_胸ポケット_裏上_正常_まっすぐ帰宅',
                                                  title_o ='01_02_01_4F実験室_XperiaZ3_胸ポケット_裏上_異常_まっすぐ外出',
                                                  before=6, after=0, column=column)
    
    knnModel = EstimatorKNN()

    data_n = [d.flatten() for d in data_n]
    data_o = [d.flatten() for d in data_o]
    
    knnModel.fit(data_n)
    score = knnModel.score(data_n)
    print(score)
    score = knnModel.score(data_o)
    print(score)

eval01()

path = data/raw/01_0[12]_01_*
data/raw/01_02_01_4F実験室_XperiaZ3_胸ポケット_裏上_異常_まっすぐ外出
...
data/raw/01_01_01_4F実験室_XperiaZ3_胸ポケット_裏上_正常_まっすぐ帰宅
...
extracted 3 dataframes
extracted 3 dataframes
10.0123495706
196.649984915
